In [5]:
import os
import time
import json
from confluent_kafka import Producer, Consumer, TopicPartition
from web3 import Web3
from hexbytes import HexBytes
from web3.datastructures import AttributeDict


# -----------------------------
# Environment Variables
# -----------------------------
ETH_RPC_URL = os.getenv("ETH_RPC_URL")
ETHERSCAN_API_KEY = os.getenv("ETHERSCAN_API_KEY")
KAFKA_BROKER = os.getenv("KAFKA_BROKER", "kafka-kafka-brokers.kafka.svc.cluster.local:9092")

TOPIC_BLOCK = os.getenv("TOPIC_BLOCK", "eth-blocks")
STATE_TOPIC = os.getenv("STATE_TOPIC", "eth-ingestion-state")
STATE_KEY = os.getenv("STATE_KEY", "eth-mainnet")

BATCH_SIZE = int(os.getenv("BATCH_SIZE", "50"))

START_BLOCK = os.getenv("START_BLOCK")
END_BLOCK = os.getenv("END_BLOCK")

START_DATE = os.getenv("START_DATE")
END_DATE = os.getenv("END_DATE")

In [6]:
# -----------------------------
# read last_block from Kafka compact topic
# -----------------------------
def load_last_block_from_kafka():
    consumer = Consumer({
        "bootstrap.servers": KAFKA_BROKER,
        "group.id": f"state-reader-{STATE_KEY}",
        "auto.offset.reset": "earliest",
        "enable.auto.commit": False,
    })

    consumer.assign([TopicPartition(STATE_TOPIC, 0)])

    last_block = None
    while True:
        msg = consumer.poll(1.0)
        if msg is None:
            break
        if msg.error():
            continue

        if msg.key() and msg.key().decode() == STATE_KEY:
            last_block = json.loads(msg.value())["last_block"]

    consumer.close()
    return last_block

In [10]:
last_block = load_last_block_from_kafka()
print(last_block)

24157782
